# Quantum Signal Processing for Hamiltonian Simulation

The script implements $e^{-i\tau H}$ using quantum signal processing (QSP) and a simple linear combination of unitaries (LCU) wrapper. 

The workflow proceeds as follows:

1. **Scale the Hamiltonian**: $H_s = H / \alpha$, with $\alpha = \max_i |\lambda_i(H)|$, so $\operatorname{spec}(H_s) \subseteq [-1,1]$.
2. **Define Targets**: Build Chebyshev approximants to $0.5\cos(Tx)$ and $0.5\sin(Tx)$ on $x\in[-1,1]$, where $T=\tau\alpha$.
3. **Synthesize phases**: Use QSP to find phase lists (even for cosine, odd for sine).
4. **Block-encoding** (Halmos) and **reflection**: Build a unitary $U_A$ with top-left block $H_s$ and use $U_\Pi=Z\otimes I$.
5. **QSP products**: Build $U_{\cos}$ and $U_{\sin}$ from phase lists.
6. **LCU combine**: Interfere controlled $(U_{\cos},U_{\sin})$ to get $\tfrac12(U_{\cos}-iU_{\sin})$; project ancillas to obtain $\tfrac14 e^{-i\tau H}$.



In [6]:
# ---------- Imports ----------

import numpy as np
import os
from numpy import kron
from numpy.linalg import eigvals, norm
from scipy.linalg import expm, sqrtm
from scipy.io import loadmat  

In [7]:
# ---------- Load phases and Hamiltonian ----------

mat_path = r"C:\Users\CristinaÁlvarezYuste\qsp_exports\qsp_phases_tau50.mat"
assert os.path.exists(mat_path), f"Missing file: {mat_path}"
data = loadmat(mat_path, squeeze_me=True, struct_as_record=False)

phi_even = np.asarray(data['phi_even'], dtype=float).ravel() 
phi_odd  = np.asarray(data['phi_odd'],  dtype=float).ravel()

meta = data['meta']
print(f"\nLoaded phases: len(phi_even) = {len(phi_even)}, len(phi_odd) = {len(phi_odd)}")

H = np.array(data['H'], dtype=complex)
tau   = float(meta.tau); 
alpha = float(meta.alpha); 
T = float(meta.T)
Hs = H / alpha
lam_Hs = eigvals(Hs)
lam_H = eigvals(H)

print(f"\nEffective time T = tau * alpha = {tau:.6f} * {alpha:.6f} = {T:.6f}")
print(f"\nScaled Hamiltonian Hs = H / alpha:\n{np.round(Hs, 2)}")
print(f"\nalpha = Spectral radius ||H||_spec ≈ {alpha:.6f}")
print(f"\nmax |eig(H)| = {np.max(np.abs(lam_H)):.6f}  (should match alpha)")
print(f"max |eig(Hs)| = {np.max(np.abs(lam_Hs)):.6f}  (should be ≤ 1)")



Loaded phases: len(phi_even) = 79, len(phi_odd) = 80

Effective time T = tau * alpha = 50.000000 * 0.742024 = 37.101189

Scaled Hamiltonian Hs = H / alpha:
[[ 0.54+0.j    0.  +0.j    0.54+0.j    0.  -0.27j]
 [ 0.  +0.j   -0.54+0.j    0.  -0.27j  0.54+0.j  ]
 [ 0.54+0.j    0.  +0.27j  0.27+0.j    0.  +0.j  ]
 [ 0.  +0.27j  0.54+0.j    0.  +0.j   -0.27+0.j  ]]

alpha = Spectral radius ||H||_spec ≈ 0.742024

max |eig(H)| = 0.742024  (should match alpha)
max |eig(Hs)| = 1.000000  (should be ≤ 1)


## Quantum Signal Processing Unitary

Given a block-encoding $U_A$ of the (scaled) operator and the reflection
$U_\Pi = 2\Pi - I$, QSP applies an alternating sequence of reflections and signal unitaries parameterized by phases. The $d$-iterate, $(d{+}1)$-phase circuit is:

$$
U_{\tilde{\Phi}}
=\Bigg[\prod_{i=0}^{d-1}\Big(e^{\,i\tilde{\phi}_i\,U_\Pi}\; U_\Pi U_A\Big)\Bigg]\;
e^{\,i\tilde{\phi}_d\,U_\Pi}
$$

Here each $\tilde{\phi}_i \in \mathbb{R}$, and $\tilde{\Phi}=(\tilde{\phi}_0,\ldots,\tilde{\phi}_d)$ is the phase vector that
encodes the target polynomial $f(x)$.


In [8]:
# ---------- Functions ----------
def qsp_unitary(UA: np.ndarray, UPi: np.ndarray, phi: np.ndarray) -> np.ndarray:
    """
    U_{~Phi} = [∏_{i=0}^{d-1} (e^{i phi_i UPi} UPi UA)] e^{i phi_d UPi}
    """
    d = len(phi) - 1
    U = np.eye(UA.shape[0], dtype=complex)
    for i in range(d):                           # i = 0,1,...,d-1
        U = U @ (expm(1j*phi[i]*UPi) @ (UPi @ UA))
    U = U @ expm(1j*phi[d]*UPi)                  # final factor on the right
    return U

## Preparing the BE of $H_s$ ($U_A$) and Reflection ($U_\Pi$)

$H_s$ must be embeded inside a unitary so QSP can act on it, this time only for numerical validation, the Halmos BE is used to construct $U_A$:
$$
U_A = \begin{bmatrix} H_s & \sqrt{I - H_s^2} \\ \sqrt{I - H_s^2} & -H_s\end{bmatrix},
\qquad
U_\Pi = Z\otimes I,
$$
such that
$$
(\langle 0 \rvert \otimes I) \, U_A \, (\lvert 0 \rangle \otimes I) = H_s
$$

The operator $U_\Pi$ is the reflection about the ancilla-$\ket{0}$ subspace:

$$
U_\Pi = 2 \Pi - I, \quad
\Pi = \ket{0}\bra{0} \otimes I
\quad \Rightarrow \quad
U_\Pi = Z_\text{anc} \otimes I = \text{diag}(1,-1) \otimes I.
$$

In circuits, $e^{i \phi U_\Pi}$ is just a $Z$-rotation on the ancilla (up to a global phase).



**Register structure:** Moving from $H_s$ to $U_A$ introduces one ancilla qubit $a$. The total Hilbert space is now $a \otimes \text{system}$ with dimension $2 \times n = 2n$.

**Well-posedness:** The matrix square root $\sqrt{I - H_s^2}$ is well-defined and positive semidefinite when $H_s$ is Hermitian with $\|H_s\| \le 1$ (which holds after spectral scaling).

In [9]:
# ---------- Block-encoding (Halmos only for testing) + reflection ----------
n  = Hs.shape[0]
Rt = sqrtm(np.eye(n) - Hs @ Hs)                         # Off diagonal block is PSD because ||Hs|| ≤ 1
UA = np.block([[Hs, Rt], [Rt, -Hs]])                    # Simple Halmos dilation (2n x 2n)
UPi = kron(np.diag([1, -1]).astype(complex), np.eye(n, dtype=complex))  # Z on ancilla a

print("||UA^*UA - I|| =", norm(UA.conj().T @ UA - np.eye(2*n))) # should be ≈ 0 (numerically unitary)
print("\nThe system Hamiltonian Hs has dimension:", Hs.shape)
print( "The system Hamiltonian Hs is block-encoded in UA with dimension:", UA.shape)

||UA^*UA - I|| = 9.787475296204779e-08

The system Hamiltonian Hs has dimension: (4, 4)
The system Hamiltonian Hs is block-encoded in UA with dimension: (8, 8)


## QSP product formula and implementation
With phase list $\tilde{\Phi}=(\tilde{\phi}_0,\ldots,\tilde{\phi}_d)$, we implement
$$
U_{\tilde{\Phi}}
=\Bigg[\prod_{i=0}^{d-1}\Big(e^{\,i\tilde{\phi}_i\,U_\Pi}\; U_\Pi U_A\Big)\Bigg]\;
e^{\,i\tilde{\phi}_d\,U_\Pi}.
$$
We build $U_{\cos}=U_{\tilde{\Phi}_{\cos}}$ and $U_{\sin}=U_{\tilde{\Phi}_{\sin}}$.


In [10]:
# ---------- QSP unitaries ----------
# These two QSP products implement polynomials of the eigenvalues x \in [-1,1] of Hs:

Ucos = qsp_unitary(UA, UPi, phi_even)   # 1/2 cos(T*x) --> <0|Ucos|0> = 1/2 cos(T*Hs)
Usin = qsp_unitary(UA, UPi, phi_odd)    # 1/2 sin(T*x) --> <0|Usin|0> = 1/2 sin(T*Hs)

print("QSP unitaries:")
print(f"Ucos  shape = {Ucos.shape},  ||Ucos|| = {norm(Ucos, 2):.6f}")
print(f"Usin  shape = {Usin.shape},  ||Usin|| = {norm(Usin, 2):.6f}")

QSP unitaries:
Ucos  shape = (8, 8),  ||Ucos|| = 1.000002
Usin  shape = (8, 8),  ||Usin|| = 1.000002


## LCU combination with a second ancilla $c$

We already have two QSP unitaries that act on **(ancilla a ⊗ system)** of size $2n\times 2n$:
- $U_{\cos}$ implements $\langle a{=}0|\,U_{\cos}\,|a{=}0\rangle \approx \tfrac12 \cos(T H_s)$
- $U_{\sin}$ implements $\langle a{=}0|\,U_{\sin}\,|a{=}0\rangle \approx \tfrac12 \sin(T H_s)$

To **linearly combine** these into $ \tfrac12\big(U_{\cos} - i\,U_{\sin}\big) $, we introduce a *second* one-qubit ancilla $c$ (the “control” register). The total space is now:
$$
\text{control }c\ (2)\ \otimes\ \big(\text{ancilla }a\ (2)\ \otimes\ \text{system }(n)\big)
\quad\Rightarrow\quad 4n \times 4n.
$$


We will always order registers as $(c, a, \text{system})$. In code, that’s why you see kron(thing_on_c, thing_on_as).


### Controlled selection: acts on $c$ ⊗  ($a$ $\otimes$ system):
$$
U_{\text{controlled}} \;=\; P_0 \otimes U_{\cos} \;+\; P_1 \otimes U_{\sin}.
$$
This means: if $c=0$ then apply $U_{\cos}$, if $c=1$ then apply $U_{\sin}$

### Interference to produce the linear combination:

Starting with control $|0\rangle_c$, apply
$$
U_{\text{total}} \;=\; (H_c \otimes I_{2n})\;\big( S_c^\dagger \otimes I_{2n} \big)\;U_{\text{controlled}}\;\big(H_c \otimes I_{2n}\big).
$$

if you post-select or project the control onto $|0\rangle_c$, the effective operator acting on $(a\otimes \text{system})$ is exactly
$$
B \;=\; \langle 0|_c\,U_{\text{total}}\,|0\rangle_c
\;=\; \tfrac12\Big(U_{\cos} \;-\; i\,U_{\sin}\Big).
$$

### Extract the system action:

Now project ancilla $a$ onto $|0\rangle$ by taking the **top-left $n\times n$** block:
$$
B_{00}\;=\;\langle 0|_a\, B \,|0\rangle_a
\;\approx\; \tfrac12\!\left(\tfrac12\cos(T H_s) \;-\; i\,\tfrac12\sin(T H_s)\right)
\;=\; \tfrac14\,e^{-i T H_s}
\;=\; \tfrac14\,e^{-i \tau H}.
$$

Because our cosine/sine targets were scaled by $1/2$, and the LCU adds another $1/2$, we multiply by 4 at the end to recover the full propagator:
$$
U_{\text{from QSP}} \;=\; 4\, B_{00}\ \approx\ e^{-i\tau H}.
$$



In [11]:
# ---------- LCU combine with a second ancilla c ----------
Hc = (1/np.sqrt(2))*np.array([[1,1],[1,-1]], dtype=complex)  # Hadamard on control c
Sd = np.diag([1, -1j])                                       # S^\dagger on control c
P0 = np.array([[1,0],[0,0]], dtype=complex)                  # |0><0| on control c
P1 = np.array([[0,0],[0,1]], dtype=complex)                  # |1><1| on control c

U_controlled = kron(P0, Ucos) + kron(P1, Usin) # c-controlled on c
U_total = kron(Hc, np.eye(2*n)) @ (kron(Sd, np.eye(2*n)) @ U_controlled) @ kron(Hc, np.eye(2*n)) # (4n x 4n)

# Project c→|0>, then a→|0> (top-left blocks)
B   = U_total[:2*n, :2*n]   # <c=0|U_total|c=0>
B00 = B[:n, :n]             # <a=0|   B   |a=0>

# Undo the (1/2)*(1/2) scaling: B00 ≈ e^{-i τ H}/4
U_from_QSP = 4 * B00
U_exact    = expm(-1j * tau * H)

relFro  = norm(U_from_QSP - U_exact, 'fro') / max(norm(U_exact, 'fro'), 1e-16)
fidelity = np.abs(np.trace(U_from_QSP.conj().T @ U_exact)) / n
print(f"LCU + QSP vs expm: {relFro:.3e}, fidelity: {fidelity:.6f}")

LCU + QSP vs expm: 2.173e+00, fidelity: 1.149460
